In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from art.estimators.classification import KerasClassifier
from art.attacks.inference.model_inversion import MIFace

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

import cv2
import os

2024-03-29 23:23:38.412464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 23:23:39.098080: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()

In [3]:
def load_att_dataset(directory):
    X, y = [], []
    for label, subdir in enumerate(sorted(os.listdir(directory))):
        subdir_path = os.path.join(directory, subdir)
        if not os.path.isdir(subdir_path):
            continue
        for filename in os.listdir(subdir_path):
            filepath = os.path.join(subdir_path, filename)
            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            X.append(image)
            y.append(label)
    X = np.array(X, dtype='float32')
    X = np.expand_dims(X, axis=-1) / 255.0  # Normalize and add channel dimension
    y = to_categorical(np.array(y))
    return X, y

X, y = load_att_dataset('at_and_t')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [8]:
classifier = KerasClassifier(clip_values=(0, 1), model=model, use_logits=False)

# Configure MIFace parameters as per your requirement
attack = MIFace(classifier, steps=1000, batch_size=1)

# Here we demonstrate the attack on a single target label
target = np.array([1])  # Example target label, adjust as needed
reconstructed_images = attack.infer(x=None, y=target)

# Display the reconstructed image
plt.imshow(reconstructed_images[0].squeeze(), cmap='gray')
plt.title('Reconstructed Image')
plt.show()


AttributeError: module 'tensorflow.keras.backend' has no attribute 'placeholder'

In [9]:
tf.version

<module 'tensorflow._api.v2.version' from '/home/bob/anaconda3/envs/bob/lib/python3.12/site-packages/tensorflow/_api/v2/version/__init__.py'>